In [ ]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
from tensorflow.keras.applications import MobileNet, EfficientNetB0, ResNet50
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [ ]:
image_folder = './Training/training_words'
label_file = './Training/training_labels.csv'
img_folder_test = './Testing/testing_words'
label_test = './Testing/testing_labels.csv'
img_val = './Validation/validation_words'
label_val = './Validation/validation_labels.csv'

In [ ]:
labels_df_train = pd.read_csv(label_file)
label_test = pd.read_csv(label_test)
label_val = pd.read_csv(label_val)


In [54]:

print("First few rows of the dataframe:")
print(labels_df_train.head())

First few rows of the dataframe:
   IMAGE MEDICINE_NAME GENERIC_NAME
0  0.png         Aceta  Paracetamol
1  1.png         Aceta  Paracetamol
2  2.png         Aceta  Paracetamol
3  3.png         Aceta  Paracetamol
4  4.png         Aceta  Paracetamol


In [55]:
len(labels_df_train['MEDICINE_NAME'].unique())

78

In [62]:
data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
  ]
)

In [149]:

def load_data(image_dir, csv_file, img_width=128, img_height=32):
    data = pd.read_csv(csv_file)
    
    images = []
    labels = []

    for index, row in data.iterrows():
        img_path = os.path.join(image_dir, row['IMAGE'])
        if os.path.exists(img_path):
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if image is not None:
                image = cv2.resize(image, (img_width, img_height))
                image = image / 255.0 
                images.append(image)
                labels.append(row['MEDICINE_NAME'])

    images = np.array(images).reshape(-1, img_height, img_width, 1)
    return images, labels

train_images, train_labels = load_data(image_folder, label_file)
validation_images, validation_labels = load_data(img_val, label_val)
test_images, test_labels = load_data(img_folder_test, label_test)

In [150]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
all_labels = train_labels + validation_labels + test_labels
label_encoder.fit(all_labels)

train_labels_encoded = label_encoder.transform(train_labels)
validation_labels_encoded = label_encoder.transform(validation_labels)
test_labels_encoded = label_encoder.transform(test_labels)

In [151]:
train_images = train_images.reshape((-1, 32, 128, 1))  # Update height and width as per your data
validation_images = validation_images.reshape((-1, 32, 128, 1))  # Same here
test_images= test_images.reshape((-1, 32, 128, 1)) 
train_labels1 = np.array(train_labels)
validation_labels1 =np.array(validation_labels)
test_labels=np.array(test_labels)
train_images = np.array(train_images)
train_labels = np.array(train_labels_encoded)
validation_images=np.array(validation_images)
validation_labels =np.array(validation_labels_encoded)
test_images=np.array(test_images)
test_labels=np.array(test_labels_encoded)
test_images.shape

(780, 32, 128, 1)

In [152]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def create_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.4)(x)

    # Compute the output shape to determine the reshape dimensions
    output_shape = x.shape[1:]  
    reshape_dim = output_shape[0] * output_shape[1]  

   
    x = layers.Reshape((reshape_dim, 256))(x)  

    x = layers.Bidirectional(layers.LSTM(256, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.01), recurrent_dropout=0.2))(x)
   # x = layers.Bidirectional(layers.LSTM(256, kernel_regularizer=tf.keras.regularizers.l2(0.01), recurrent_dropout=0.2))(x)

    
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model
from tensorflow.keras.optimizers import Adam
input_shape = (32, 128, 1)  # Height, width, channels
num_classes = len(label_encoder.classes_)
model = create_model(input_shape, num_classes)
# Compile the model

model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [153]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_callback = ModelCheckpoint('model_best.keras', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

In [154]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

history = model.fit(
    train_images, train_labels,
    validation_data=(validation_images, validation_labels),
    epochs=100, 
    batch_size=32,
    callbacks=[early_stopping, reduce_lr,checkpoint_callback]
)

Epoch 1/100
98/98 ━━━━━━━━━━━━━━━━━━━━ 42s 337ms/step - accuracy: 0.0377 - loss: 17.1742 - learning_rate: 0.0010
Epoch 2/100
98/98 ━━━━━━━━━━━━━━━━━━━━ 32s 325ms/step - accuracy: 0.1049 - loss: 4.6544 - learning_rate: 0.0010
Epoch 3/100
98/98 ━━━━━━━━━━━━━━━━━━━━ 32s 324ms/step - accuracy: 0.2172 - loss: 3.8185 - learning_rate: 0.0010
Epoch 4/100
98/98 ━━━━━━━━━━━━━━━━━━━━ 32s 323ms/step - accuracy: 0.3049 - loss: 3.4540 - learning_rate: 0.0010
Epoch 5/100
98/98 ━━━━━━━━━━━━━━━━━━━━ 32s 323ms/step - accuracy: 0.3851 - loss: 3.2801 - learning_rate: 0.0010
Epoch 6/100
98/98 ━━━━━━━━━━━━━━━━━━━━ 32s 327ms/step - accuracy: 0.4932 - loss: 2.9150 - learning_rate: 0.0010
Epoch 7/100
98/98 ━━━━━━━━━━━━━━━━━━━━ 32s 326ms/step - accuracy: 0.5750 - loss: 2.6580 - learning_rate: 0.0010
Epoch 8/100
98/98 ━━━━━━━━━━━━━━━━━━━━ 32s 327ms/step - accuracy: 0.6121 - loss: 2.6594 - learning_rate: 0.0010
Epoch 9/100
98/98 ━━━━━━━━━━━━━━━━━━━━ 32s 322ms/step - accuracy: 0.6539 - loss: 2.4866 - learning_rate

In [156]:
test_loss, test_accuracy = model.evaluate(test_images, test_labels, batch_size=32)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.5203 - loss: 3.2280
Test Loss: 3.738107919692993
Test Accuracy: 0.47564101219177246
